# **Market Sentiment & Trader Performance: A 2024 Deep Dive**
Author:
Harshil Patel

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# load data
historical_data = pd.read_csv('../data/historical_data.csv')
fear_greed_index = pd.read_csv('../data/fear_greed_index.csv')

In [3]:
# Basic info
print("Historical Data:")
print(f"Rows: {len(historical_data)}")
print(f"Columns: {len(historical_data.columns)}")
print(f"Column names: {list(historical_data.columns)}")

Historical Data:
Rows: 211224
Columns: 16
Column names: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']


In [4]:
print("Sentiment Data:")
print(f"Rows: {len(fear_greed_index)}")
print(f"Columns: {len(fear_greed_index.columns)}")
print(f"Column names: {list(fear_greed_index.columns)}")

Sentiment Data:
Rows: 2644
Columns: 4
Column names: ['timestamp', 'value', 'classification', 'date']


In [5]:
historical_data.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [6]:
historical_data.sort_values(by='Timestamp IST', ascending=False)

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
135705,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,@107,25.45200,10.00000,254.52,BUY,31-12-2024 23:33,20.074135,Buy,0.0,0x53c93c8be3ac82e85ff4041a54519201fa005e0a9daa...,59875521503,True,0.003500,1.060000e+15,1.740000e+12
135704,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,PURR/USDC,0.28724,138.00000,39.64,BUY,31-12-2024 23:31,40355.183720,Buy,0.0,0x742981ec1ecc4b4e9db2041a544a5102014c00ca9956...,59875041028,True,0.048290,1.090000e+15,1.740000e+12
135703,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,PURR/USDC,0.28715,150.00000,43.07,BUY,31-12-2024 23:31,40205.236210,Buy,0.0,0x742981ec1ecc4b4e9db2041a544a5102014c00ca9956...,59875041028,True,0.052490,6.620000e+13,1.740000e+12
135702,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,PURR/USDC,0.28715,234.00000,67.19,BUY,31-12-2024 23:31,39971.318100,Buy,0.0,0x742981ec1ecc4b4e9db2041a544a5102014c00ca9956...,59875041028,True,0.081890,8.480000e+14,1.740000e+12
135698,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,BTC,94213.00000,0.01100,1036.34,BUY,31-12-2024 23:29,0.127440,Open Long,0.0,0x1c13b9c6022729e54cb4041a54434102019000a77395...,59874607028,True,0.362720,7.030000e+14,1.740000e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39399,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,ETH,2299.80000,1.12500,2587.28,BUY,01-01-2024 01:23,0.697700,Open Long,0.0,0xa2936c42063c5717b7e70406e834fc010700a01cd9a4...,5879941423,True,0.646818,1.030000e+15,1.700000e+12
39398,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,ETH,2299.70000,0.01270,29.21,BUY,01-01-2024 01:23,0.685000,Open Long,0.0,0xa2936c42063c5717b7e70406e834fc010700a01cd9a4...,5879941423,True,0.007301,1.370000e+14,1.700000e+12
39397,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,ETH,2299.70000,0.43500,1000.37,BUY,01-01-2024 01:23,0.250000,Open Long,0.0,0xa2936c42063c5717b7e70406e834fc010700a01cd9a4...,5879941423,True,0.250092,2.120000e+13,1.700000e+12
39396,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,ETH,2299.50000,0.25000,574.88,BUY,01-01-2024 01:23,0.000000,Open Long,0.0,0xa2936c42063c5717b7e70406e834fc010700a01cd9a4...,5879941423,True,0.143718,7.930000e+14,1.700000e+12


In [7]:
fear_greed_index.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [8]:
historical_data['Account'].nunique()

32

In [9]:
fear_greed_index['classification'].unique()

array(['Fear', 'Extreme Fear', 'Neutral', 'Greed', 'Extreme Greed'],
      dtype=object)

In [10]:
fear_greed_index['value'].min()

np.int64(5)

In [11]:
# For Historical Data - check date range
print("HISTORICAL DATA - Time Range:")
print(f"First trade: {historical_data['Timestamp IST'].min()}")
print(f"Last trade: {historical_data['Timestamp IST'].max()}")

HISTORICAL DATA - Time Range:
First trade: 01-01-2024 01:23
Last trade: 31-12-2024 23:33


In [12]:
# For sentiment Data - check date range
print("SENTIMENT DATA - Time Range:")
print(f"First trade: {fear_greed_index['date'].min()}")
print(f"Last trade: {fear_greed_index['date'].max()}")

SENTIMENT DATA - Time Range:
First trade: 2018-02-01
Last trade: 2025-05-02


In [13]:
# trades per day on average
print("HISTORICAL DATA SCALE:")
print(f"Total trades: {len(historical_data):,}")
print(f"Days covered: ~365 (full year 2024)")
print(f"Average trades per day: ~{len(historical_data) / 365:.0f}")

HISTORICAL DATA SCALE:
Total trades: 211,224
Days covered: ~365 (full year 2024)
Average trades per day: ~579


In [14]:
# unique values
print("UNIQUE VALUES:")
print("Unique Accounts: ",historical_data['Account'].nunique())
print("Unique coins traded: ",historical_data['Coin'].nunique())

UNIQUE VALUES:
Unique Accounts:  32
Unique coins traded:  246


In [15]:
# overall trading performace
print("OVERALL TRADING PERFORMANCE:")
total_trades = len(historical_data)
profitable_trades = (historical_data['Closed PnL'] > 0).sum()
losing_trades = (historical_data['Closed PnL'] < 0).sum()
breakeven_trades = (historical_data['Closed PnL'] == 0).sum()

print(f"Profitable trades: {profitable_trades:,} ({profitable_trades/total_trades*100:.1f}%)")
print(f"Losing trades: {losing_trades:,} ({losing_trades/total_trades*100:.1f}%)")
print(f"Break-even trades: {breakeven_trades:,} ({breakeven_trades/total_trades*100:.1f}%)")

OVERALL TRADING PERFORMANCE:
Profitable trades: 86,869 (41.1%)
Losing trades: 17,539 (8.3%)
Break-even trades: 106,816 (50.6%)


In [16]:
# sentiment distribution
print("SENTIMENT DISTRIBUTION:")
print(fear_greed_index['classification'].value_counts().sort_values(ascending=False))

SENTIMENT DISTRIBUTION:
classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64


In [17]:
# groupby account to see the account wise trade analysis
account_performance=historical_data.groupby("Account")

account_performance_df=pd.DataFrame({
    'Total Trades': account_performance.size(),
    'Profitable Trades': account_performance.apply(lambda x: (x['Closed PnL'] > 0).sum()),
    'Losing Trades': account_performance.apply(lambda x: (x['Closed PnL'] < 0).sum()),
    'Breakeven Trades': account_performance.apply(lambda x: (x['Closed PnL'] == 0).sum())
})

account_performance_df.sort_values(by='Total Trades', ascending=False).head()

,Total Trades,Profitable Trades,Losing Trades,Breakeven Trades
Account,,,,
0xbee1707d6b44d4d52bfe19e41f8a828645437aab,40184,17208,5343,17633
0xbaaaf6571ab7d571043ff1e313a9609a10637864,21192,9909,88,11195
0xa0feb3725a9335f49874d7cd8eaad6be45b27416,15605,5396,165,10044
0x8477e447846c758f5a675856001ea72298fd9cb5,14998,3929,2630,8439
0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,14733,4967,1312,8454


In [18]:
coin_grouped = historical_data.groupby("Coin")
coin_performance_df=pd.DataFrame({
    'Total Trades': coin_grouped.size(),
    'Profitable Trades': coin_grouped.apply(lambda x: (x['Closed PnL'] > 0).sum()),
    'Losing Trades': coin_grouped.apply(lambda x: (x['Closed PnL'] < 0).sum()),
    'Breakeven Trades': coin_grouped.apply(lambda x: (x['Closed PnL'] == 0).sum())
})
coin_performance_df.sort_values(by='Total Trades', ascending=False).head(10)  

,Total Trades,Profitable Trades,Losing Trades,Breakeven Trades
Coin,,,,
HYPE,68005,28222,3789,35994
@107,29992,14024,3142,12826
BTC,26064,9144,1865,15055
ETH,11158,4016,1212,5930
SOL,10691,4219,811,5661
FARTCOIN,4650,1527,637,2486
MELANIA,4428,1966,245,2217
PURR/USDC,2774,989,198,1587
WLD,1983,711,229,1043


In [19]:
print("TRADE OUTCOME BREAKDOWN:")
total = len(historical_data)
wins = (historical_data['Closed PnL'] > 0).sum()
losses = (historical_data['Closed PnL'] < 0).sum()
breakeven = (historical_data['Closed PnL'] == 0).sum()

print(f"Wins: {wins:,} ({wins/total*100:.1f}%)")
print(f"Losses: {losses:,} ({losses/total*100:.1f}%)")
print(f"Breakeven: {breakeven:,} ({breakeven/total*100:.1f}%)")
print(f"Total: {total:,}")

print("\n" + "="*50 + "\n")

# Overall profitability
total_pnl = historical_data['Closed PnL'].sum()
print(f"TOTAL PnL across all trades: ${total_pnl:,.2f}")
print(f"Average PnL per trade: ${total_pnl/total:,.2f}")

if total_pnl > 0:
    print("✓ Overall PROFITABLE")
else:
    print("✗ Overall LOSING")

TRADE OUTCOME BREAKDOWN:
Wins: 86,869 (41.1%)
Losses: 17,539 (8.3%)
Breakeven: 106,816 (50.6%)
Total: 211,224


TOTAL PnL across all trades: $10,296,958.94
Average PnL per trade: $48.75
✓ Overall PROFITABLE


In [20]:
# First, let's see exactly how much volume each coin represents
print("TOP 10 COINS BY TRADE COUNT:")
top_coins = historical_data['Coin'].value_counts().head(10)
print(top_coins)
print("\n")

# Calculate percentage
for coin, count in top_coins.items():
    pct = (count / len(historical_data)) * 100
    print(f"{coin}: {count:,} trades ({pct:.1f}%)")

TOP 10 COINS BY TRADE COUNT:
Coin
HYPE         68005
@107         29992
BTC          26064
ETH          11158
SOL          10691
FARTCOIN      4650
MELANIA       4428
PURR/USDC     2774
WLD           1983
SUI           1979
Name: count, dtype: int64


HYPE: 68,005 trades (32.2%)
@107: 29,992 trades (14.2%)
BTC: 26,064 trades (12.3%)
ETH: 11,158 trades (5.3%)
SOL: 10,691 trades (5.1%)
FARTCOIN: 4,650 trades (2.2%)
MELANIA: 4,428 trades (2.1%)
PURR/USDC: 2,774 trades (1.3%)
WLD: 1,983 trades (0.9%)
SUI: 1,979 trades (0.9%)


In [21]:
# 1. Clean the date column
fear_greed_index['date'] = (
    fear_greed_index['date']
        .astype(str)     # ensure string
        .str.strip()     # remove trailing/leading spaces
        .str.replace(r'\s+', ' ', regex=True)  # collapse weird whitespace
)

# 2. Parse using mixed formats
fear_greed_index['date'] = pd.to_datetime(
    fear_greed_index['date'],
    format='mixed',
    errors='coerce'   # invalid values → NaT instead of crashing
)

# 3. Check unparsed dates
bad_dates = fear_greed_index[fear_greed_index['date'].isna()]
print("Unparsed date rows:", len(bad_dates))


Unparsed date rows: 0


In [22]:
# Convert sentiment date to datetime
fear_greed_index['date'] = pd.to_datetime(fear_greed_index['date'])

# Filter for 2024 only
sentiment_2024 = fear_greed_index[(fear_greed_index['date'] >= '2024-01-01') & 
                           (fear_greed_index['date'] <= '2024-12-31')].copy()
print(f"Original sentiment data: {len(fear_greed_index)} rows")
print(f"Sentiment data for 2024: {len(sentiment_2024)} rows")
print(f"Expected for 2024 (leap year): 366 days")

# Check sentiment distribution in 2024
print("\nSentiment in 2024:")
print(sentiment_2024['classification'].value_counts())

Original sentiment data: 2644 rows
Sentiment data for 2024: 365 rows
Expected for 2024 (leap year): 366 days

Sentiment in 2024:
classification
Greed            161
Extreme Greed    102
Fear              55
Neutral           43
Extreme Fear       4
Name: count, dtype: int64


In [23]:
# so 2024 is leap year so there should be 366 days of data which one missing so we can check which date is missing
complete_2024 = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')

# Find missing dates
existing_dates = set(sentiment_2024['date'])
missing_dates = set(complete_2024) - existing_dates

if missing_dates:
    print(f"Missing date(s): {missing_dates}")
else:
    print("No missing dates!")

Missing date(s): {Timestamp('2024-10-26 00:00:00')}


In [24]:
historical_data['Timestamp IST'].tail(10)

211214    25-04-2025 15:35
211215    25-04-2025 15:35
211216    25-04-2025 15:35
211217    25-04-2025 15:35
211218    25-04-2025 15:35
211219    25-04-2025 15:35
211220    25-04-2025 15:35
211221    25-04-2025 15:35
211222    25-04-2025 15:35
211223    25-04-2025 15:35
Name: Timestamp IST, dtype: object

In [25]:
# extract only date from the timestamp IST

# Convert Timestamp IST to datetime
historical_data['Timestamp IST'] = pd.to_datetime(historical_data['Timestamp IST'], 
                                               format='%d-%m-%Y %H:%M')

# Extract just the date
historical_data['date_only'] = historical_data['Timestamp IST'].dt.date

# Convert back to datetime for easier merging
historical_data['date_only'] = pd.to_datetime(historical_data['date_only'])

# Verify it worked
print("Sample of date conversion:")
print(historical_data[['Timestamp IST', 'date_only']].head())

Sample of date conversion:
        Timestamp IST  date_only
0 2024-12-02 22:50:00 2024-12-02
1 2024-12-02 22:50:00 2024-12-02
2 2024-12-02 22:50:00 2024-12-02
3 2024-12-02 22:50:00 2024-12-02
4 2024-12-02 22:50:00 2024-12-02


In [26]:
# Check the date range
print(f"First trade date: {historical_data['date_only'].min()}")
print(f"Last trade date: {historical_data['date_only'].max()}")

# How many unique days of trading?
unique_days = historical_data['date_only'].nunique()
print(f"\nUnique trading days: {unique_days}")

First trade date: 2023-05-01 00:00:00
Last trade date: 2025-05-01 00:00:00

Unique trading days: 480


In [27]:
# Check the original Timestamp IST column
print("Original Timestamp IST - First 10 values:")
print(historical_data['Timestamp IST'].head(10))

print("\n" + "="*50 + "\n")

print("Original Timestamp IST - Data type:")
print(historical_data['Timestamp IST'].dtype)

print("\n" + "="*50 + "\n")

# Check date range in ORIGINAL column (before conversion)
print("Date range check on ORIGINAL column:")
print(f"Min: {historical_data['Timestamp IST'].min()}")
print(f"Max: {historical_data['Timestamp IST'].max()}")

Original Timestamp IST - First 10 values:
0   2024-12-02 22:50:00
1   2024-12-02 22:50:00
2   2024-12-02 22:50:00
3   2024-12-02 22:50:00
4   2024-12-02 22:50:00
5   2024-12-02 22:50:00
6   2024-12-02 22:50:00
7   2024-12-02 22:50:00
8   2024-12-02 22:50:00
9   2024-12-02 22:50:00
Name: Timestamp IST, dtype: datetime64[ns]


Original Timestamp IST - Data type:
datetime64[ns]


Date range check on ORIGINAL column:
Min: 2023-05-01 01:06:00
Max: 2025-05-01 12:13:00


In [28]:
# filter historical data for 2024 only
historical_2024 = historical_data[
    (historical_data['date_only'] >= '2024-01-01') & 
    (historical_data['date_only'] <= '2024-12-31')
].copy()

print(f"Original historical data: {len(historical_data):,} trades")
print(f"Historical data for 2024: {len(historical_2024):,} trades")

print("\n" + "="*50 + "\n")

# Check the date range
print(f"First trade in 2024: {historical_2024['date_only'].min()}")
print(f"Last trade in 2024: {historical_2024['date_only'].max()}")

print("\n" + "="*50 + "\n")

# How many unique days?
unique_days_2024 = historical_2024['date_only'].nunique()
print(f"Unique trading days in 2024: {unique_days_2024}")
print(f"Expected days in 2024: 366")

# Check if any dates are missing
date_range_2024 = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
existing_trade_dates = set(historical_2024['date_only'])
missing_trade_dates = set(date_range_2024) - existing_trade_dates

if missing_trade_dates:
    print(f"\n⚠ Days without trades: {len(missing_trade_dates)}")
    print(f"Missing dates: {sorted(missing_trade_dates)}")
else:
    print("\n✓ Trades on every day of 2024!")

Original historical data: 211,224 trades
Historical data for 2024: 52,497 trades


First trade in 2024: 2024-01-01 00:00:00
Last trade in 2024: 2024-12-31 00:00:00


Unique trading days in 2024: 341
Expected days in 2024: 366

⚠ Days without trades: 25
Missing dates: [Timestamp('2024-01-13 00:00:00'), Timestamp('2024-01-14 00:00:00'), Timestamp('2024-01-27 00:00:00'), Timestamp('2024-02-09 00:00:00'), Timestamp('2024-02-10 00:00:00'), Timestamp('2024-02-24 00:00:00'), Timestamp('2024-03-07 00:00:00'), Timestamp('2024-03-22 00:00:00'), Timestamp('2024-03-23 00:00:00'), Timestamp('2024-04-18 00:00:00'), Timestamp('2024-04-21 00:00:00'), Timestamp('2024-05-02 00:00:00'), Timestamp('2024-05-17 00:00:00'), Timestamp('2024-05-28 00:00:00'), Timestamp('2024-06-28 00:00:00'), Timestamp('2024-08-22 00:00:00'), Timestamp('2024-09-03 00:00:00'), Timestamp('2024-09-07 00:00:00'), Timestamp('2024-09-12 00:00:00'), Timestamp('2024-09-13 00:00:00'), Timestamp('2024-09-15 00:00:00'), Timestamp('2024-0

In [29]:
#  grouped sentiment column
def group_sentiment(classification):
    if classification in ['Fear','Extreme Fear']:
        return 'Bearish'
    elif classification=='Neutral':
        return 'Neutral'
    else: # greed or exrtreme greed
        return 'Bullish'
    
sentiment_2024['sentiment_group'] = sentiment_2024['classification'].apply(group_sentiment)


In [30]:
# check new distribution
print("GROUPED SENTIMENT DISTRIBUTION IN 2024:")
print(sentiment_2024['sentiment_group'].value_counts())
print("\n" + "="*50 + "\n")

# Show percentage
sentiment_pct = sentiment_2024['sentiment_group'].value_counts(normalize=True) * 100
print("GROUPED SENTIMENT PERCENTAGES:")
for sentiment, pct in sentiment_pct.items():
    print(f"{sentiment}: {pct:.1f}%")

GROUPED SENTIMENT DISTRIBUTION IN 2024:
sentiment_group
Bullish    263
Bearish     59
Neutral     43
Name: count, dtype: int64


GROUPED SENTIMENT PERCENTAGES:
Bullish: 72.1%
Bearish: 16.2%
Neutral: 11.8%


In [31]:
# crucial step now merge the dataset

# ensuring the same date type
sentiment_2024['date'] = pd.to_datetime(sentiment_2024['date']).dt.date
historical_2024['date_only'] = historical_2024['date_only'].dt.date

merged_data = historical_2024.merge(
    sentiment_2024[['date', 'value', 'classification', 'sentiment_group']], 
    left_on='date_only', 
    right_on='date', 
    how='left'
)

print("Original 2024 trades: ",len(historical_2024))
print("After merge: ",len(merged_data))
print("Trades with sentiment data: ",merged_data['sentiment_group'].notna().sum())
print("Trades without sentiment data: ",merged_data['sentiment_group'].isna().sum())
print("\n" + "="*50 + "\n")

#  sample
print("Sample of merged data:")
print(merged_data[['Timestamp IST', 'Coin', 'Side', 'Closed PnL', 
                    'sentiment_group', 'value']].head(10))

Original 2024 trades:  52497
After merge:  52497
Trades with sentiment data:  52491
Trades without sentiment data:  6


Sample of merged data:
        Timestamp IST  Coin Side  Closed PnL sentiment_group  value
0 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
1 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
2 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
3 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
4 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
5 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
6 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
7 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
8 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0
9 2024-12-02 22:50:00  @107  BUY         0.0         Bullish   80.0


In [32]:
merged_data[merged_data['sentiment_group'].isna()]


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,...,Order ID,Crossed,Fee,Trade ID,Timestamp,date_only,date,value,classification,sentiment_group
354,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,EIGEN,2.8330,10000.00,28330.00,BUY,2024-10-26 04:44:00,-32797.51,Close Short,11655.000000,...,40671411386,False,2.719680,8.350000e+13,1.730000e+12,2024-10-26,NaN,NaN,NaN,NaN
355,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,EIGEN,2.7110,10797.51,29272.05,BUY,2024-10-26 04:52:00,-22797.51,Close Short,13901.794130,...,40671438761,False,2.810115,3.230000e+14,1.730000e+12,2024-10-26,NaN,NaN,NaN,NaN
356,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,EIGEN,2.5889,6908.33,17884.98,BUY,2024-10-26 04:52:00,-12000.00,Close Short,9737.981968,...,40671466136,False,1.716957,1.070000e+15,1.730000e+12,2024-10-26,NaN,NaN,NaN,NaN
357,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,EIGEN,2.5889,529.39,1370.54,BUY,2024-10-26 04:52:00,-5091.67,Close Short,746.228144,...,40671466136,False,0.131570,8.770000e+14,1.730000e+12,2024-10-26,NaN,NaN,NaN,NaN
358,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,EIGEN,2.5889,38.65,100.06,BUY,2024-10-26 04:52:00,-4562.28,Close Short,54.481040,...,40671466136,False,0.009605,9.820000e+14,1.730000e+12,2024-10-26,NaN,NaN,NaN,NaN
359,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,EIGEN,2.5889,4523.63,11711.23,BUY,2024-10-26 04:53:00,-4523.63,Close Short,6376.508848,...,40671466136,False,1.124277,9.360000e+14,1.730000e+12,2024-10-26,NaN,NaN,NaN,NaN


## Do traders perform better during different sentiment periods?

In [33]:
print("PERFORMANCE BY SENTIMENT")
performance_by_sentiment=merged_data.groupby('sentiment_group').agg({
    'Closed PnL':['count','mean','median','sum'],
    'Trade ID':'count'
})
print(performance_by_sentiment)

PERFORMANCE BY SENTIMENT
                Closed PnL                                 Trade ID
                     count       mean median           sum    count
sentiment_group                                                    
Bearish               1451 -60.093872    0.0 -8.719621e+04     1451
Bullish              49747  71.631731    0.0  3.563464e+06    49747
Neutral               1293  68.151236    0.0  8.811955e+04     1293


In [34]:
# win rate by sentiment
for sentiment in ['Bearish','Neutral','Bullish']:
    sentiment_trades=merged_data[merged_data['sentiment_group']==sentiment]
    total=len(sentiment_trades)
    wins=(sentiment_trades['Closed PnL']>0).sum()
    win_rate=wins/total*100
    print(f"{sentiment}: {win_rate:.1f}% ({wins:,} wins out of {total:,} trades)")

print("\n" + "="*50 + "\n")


Bearish: 23.9% (347 wins out of 1,451 trades)
Neutral: 23.8% (308 wins out of 1,293 trades)
Bullish: 44.3% (22,041 wins out of 49,747 trades)




In [35]:
# total PnL by sentiment
print("TOTAL PnL BY SENTIMENT")
for sentiment in ['Bearish','Neutral','Bullish']:
    sentiment_trades=merged_data[merged_data['sentiment_group']==sentiment]
    total_pnl=sentiment_trades['Closed PnL'].sum()
    avg_pnl=sentiment_trades['Closed PnL'].mean()
    print(f"{sentiment}:")
    print(f"  Total PnL: {total_pnl:,.2f}")
    print(f"  Average PnL per trade: {avg_pnl:.4f}")
    print()


TOTAL PnL BY SENTIMENT
Bearish:
  Total PnL: -87,196.21
  Average PnL per trade: -60.0939

Neutral:
  Total PnL: 88,119.55
  Average PnL per trade: 68.1512

Bullish:
  Total PnL: 3,563,463.72
  Average PnL per trade: 71.6317



## Findings 1
- Bullish win rate is the most 44% (almost double of Bearish and Neutral)
- Means Traders are much successful duing the greed or extreame greed period

## Findings 2
- Bullish total PnL: +3,563,463 (massive profit)
- Neutral total PnL: +88,119 (BreakEven)
- Bearish total PnL: -87,196 (lossing money)

Means
- During fear, traders LOSE money
- During neutral, traders break even
- During greed, traders make BANK

## Finding 3: Average PnL Per Trade

- Bearish: -$60.09 per trade (losing)
- Neutral: +$68.15 per trade (okay)
- Bullish: +$71.63 per trade (best)

Bearish: 1,451 trades (2.8% of trades)
Neutral: 1,293 trades (2.5% of trades)
Bullish: 49,747 trades (94.7% of trades!)

There's an imbalance. But look - even the per-trade average is worse during bearish periods!

## Are Traders Trading Differently?

In [36]:
merged_data.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,...,Order ID,Crossed,Fee,Trade ID,Timestamp,date_only,date,value,classification,sentiment_group
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,2024-12-02 22:50:00,0.000000,Buy,0.0,...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,2024-12-02,2024-12-02,80.0,Extreme Greed,Bullish
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,2024-12-02 22:50:00,986.524596,Buy,0.0,...,52017706630,True,0.005600,4.430000e+14,1.730000e+12,2024-12-02,2024-12-02,80.0,Extreme Greed,Bullish
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,2024-12-02 22:50:00,1002.518996,Buy,0.0,...,52017706630,True,0.050431,6.600000e+14,1.730000e+12,2024-12-02,2024-12-02,80.0,Extreme Greed,Bullish
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,2024-12-02 22:50:00,1146.558564,Buy,0.0,...,52017706630,True,0.050043,1.080000e+15,1.730000e+12,2024-12-02,2024-12-02,80.0,Extreme Greed,Bullish
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,2024-12-02 22:50:00,1289.488521,Buy,0.0,...,52017706630,True,0.003055,1.050000e+15,1.730000e+12,2024-12-02,2024-12-02,80.0,Extreme Greed,Bullish


In [37]:
# Trading behavior by sentiment
print("="*80)
print("TRADING BEHAVIOR BY SENTIMENT")
print("="*80)

for sentiment in ['Bearish', 'Neutral', 'Bullish']:
    print(f"\n{sentiment.upper()}:")
    sentiment_data = merged_data[merged_data['sentiment_group'] == sentiment]
    
    # Position size
    avg_size = sentiment_data['Size USD'].mean()
    print(f"  Average position size: ${avg_size:,.2f}")
    
    # Buy vs Sell distribution
    side_dist = sentiment_data['Side'].value_counts()
    buy_pct = (side_dist.get('BUY', 0) / len(sentiment_data)) * 100
    sell_pct = (side_dist.get('SELL', 0) / len(sentiment_data)) * 100
    print(f"  BUY: {buy_pct:.1f}%  |  SELL: {sell_pct:.1f}%")
    
    # Top 3 coins traded
    top_coins = sentiment_data['Coin'].value_counts().head(3)
    print(f"  Top 3 coins: {', '.join(top_coins.index.tolist())}")
    
    print()

TRADING BEHAVIOR BY SENTIMENT

BEARISH:
  Average position size: $4,573.72
  BUY: 56.3%  |  SELL: 43.7%
  Top 3 coins: ETH, @4, @8


NEUTRAL:
  Average position size: $6,900.05
  BUY: 45.3%  |  SELL: 54.7%
  Top 3 coins: ETH, BTC, @4


BULLISH:
  Average position size: $3,323.81
  BUY: 44.0%  |  SELL: 56.0%
  Top 3 coins: @107, HYPE, ETH



In [38]:
# Check trader activity across sentiments
print("="*80)
print("TRADER ACTIVITY ACROSS SENTIMENTS")
print("="*80)

# For each sentiment, which traders are active?
for sentiment in ['Bearish', 'Neutral', 'Bullish']:
    sentiment_data = merged_data[merged_data['sentiment_group'] == sentiment]
    unique_traders = sentiment_data['Account'].nunique()
    total_trades = len(sentiment_data)
    
    print(f"\n{sentiment}:")
    print(f"  Active traders: {unique_traders}")
    print(f"  Total trades: {total_trades:,}")
    print(f"  Trades per trader: {total_trades/unique_traders:.0f}")

print("\n" + "="*50 + "\n")

# Are the SAME traders active across all sentiments?
bearish_traders = set(merged_data[merged_data['sentiment_group'] == 'Bearish']['Account'].unique())
neutral_traders = set(merged_data[merged_data['sentiment_group'] == 'Neutral']['Account'].unique())
bullish_traders = set(merged_data[merged_data['sentiment_group'] == 'Bullish']['Account'].unique())

all_three = bearish_traders & neutral_traders & bullish_traders
print(f"Traders active in ALL three sentiment periods: {len(all_three)}")
print(f"Traders only in Bearish: {len(bearish_traders - neutral_traders - bullish_traders)}")
print(f"Traders only in Neutral: {len(neutral_traders - bearish_traders - bullish_traders)}")
print(f"Traders only in Bullish: {len(bullish_traders - bearish_traders - neutral_traders)}")

TRADER ACTIVITY ACROSS SENTIMENTS

Bearish:
  Active traders: 7
  Total trades: 1,451
  Trades per trader: 207

Neutral:
  Active traders: 9
  Total trades: 1,293
  Trades per trader: 144

Bullish:
  Active traders: 31
  Total trades: 49,747
  Trades per trader: 1605


Traders active in ALL three sentiment periods: 6
Traders only in Bearish: 0
Traders only in Neutral: 0
Traders only in Bullish: 21


In [39]:
# Let's look at some examples
print("Sample trades to understand BUY/SELL:")
sample = merged_data[['Account', 'Coin', 'Side', 'Closed PnL', 'Size USD', 'sentiment_group']].head(20)
print(sample)

print("\n" + "="*50 + "\n")

# Check if there's a pattern
print("Profitable BUY trades:")
print(merged_data[(merged_data['Side'] == 'BUY') & (merged_data['Closed PnL'] > 0)][
    ['Coin', 'Side', 'Closed PnL', 'sentiment_group']].head(5))

print("\nProfitable SELL trades:")
print(merged_data[(merged_data['Side'] == 'SELL') & (merged_data['Closed PnL'] > 0)][
    ['Coin', 'Side', 'Closed PnL', 'sentiment_group']].head(5))

Sample trades to understand BUY/SELL:
                                       Account  Coin Side  Closed PnL  \
0   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
1   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
2   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
3   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
4   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
5   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
6   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
7   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
8   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
9   0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
10  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
11  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY         0.0   
12  0xae5eaca

In [40]:
# How many trades have zero PnL?
print("PnL DISTRIBUTION:")
zero_pnl = (merged_data['Closed PnL'] == 0).sum()
positive_pnl = (merged_data['Closed PnL'] > 0).sum()
negative_pnl = (merged_data['Closed PnL'] < 0).sum()

print(f"Zero PnL trades: {zero_pnl:,} ({zero_pnl/len(merged_data)*100:.1f}%)")
print(f"Positive PnL trades: {positive_pnl:,} ({positive_pnl/len(merged_data)*100:.1f}%)")
print(f"Negative PnL trades: {negative_pnl:,} ({negative_pnl/len(merged_data)*100:.1f}%)")

print("\n" + "="*50 + "\n")

# Let's look at NON-ZERO PnL trades only
print("PERFORMANCE BY SENTIMENT (EXCLUDING ZERO PnL):")
non_zero_data = merged_data[merged_data['Closed PnL'] != 0].copy()

for sentiment in ['Bearish', 'Neutral', 'Bullish']:
    sentiment_data = non_zero_data[non_zero_data['sentiment_group'] == sentiment]
    total = len(sentiment_data)
    wins = (sentiment_data['Closed PnL'] > 0).sum()
    win_rate = (wins / total) * 100 if total > 0 else 0
    total_pnl = sentiment_data['Closed PnL'].sum()
    avg_pnl = sentiment_data['Closed PnL'].mean()
    
    print(f"\n{sentiment}:")
    print(f"  Trades (non-zero): {total:,}")
    print(f"  Win rate: {win_rate:.1f}%")
    print(f"  Total PnL: ${total_pnl:,.2f}")
    print(f"  Avg PnL: ${avg_pnl:.2f}")

PnL DISTRIBUTION:
Zero PnL trades: 25,391 (48.4%)
Positive PnL trades: 22,702 (43.2%)
Negative PnL trades: 4,404 (8.4%)


PERFORMANCE BY SENTIMENT (EXCLUDING ZERO PnL):

Bearish:
  Trades (non-zero): 475
  Win rate: 73.1%
  Total PnL: $-87,196.21
  Avg PnL: $-183.57

Neutral:
  Trades (non-zero): 559
  Win rate: 55.1%
  Total PnL: $88,119.55
  Avg PnL: $157.64

Bullish:
  Trades (non-zero): 26,066
  Win rate: 84.6%
  Total PnL: $3,563,463.72
  Avg PnL: $136.71


# Visualization Details
1. Performance Metrics Overview (4-panel dashboard)

Total PnL by sentiment
Average PnL per trade
Win rate comparison
Trading volume

2. Risk-Reward Paradox

Distribution of wins vs losses during bearish periods
Cumulative PnL chart showing the "death by a thousand cuts"
Statistical box showing why high win rate still loses money

3. Trader Participation

Number of active traders in each sentiment period
Trading intensity (average trades per trader)
Shows the 68% who only trade during bullish

4. Position Sizing & Behavior

Counter-intuitive position sizing strategy
Buy vs Sell distribution across sentiments
Highlights contrarian behavior

5. Top Coins by Sentiment

Side-by-side comparison of coin preferences
Shows shift from majors (ETH/BTC) to meme coins (HYPE/@107)

6. Executive Dashboard

All-in-one summary view
Key metrics table
Mini charts
Text box with key insights


In [41]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# ============================================================================
# SETUP: Prepare data for visualizations
# ============================================================================

# Filter to non-zero PnL for accurate analysis
non_zero_data = merged_data[merged_data['Closed PnL'] != 0].copy()

sentiments = ['Bearish', 'Neutral', 'Bullish']
colors_dict = {'Bearish': '#e74c3c', 'Neutral': '#95a5a6', 'Bullish': '#2ecc71'}

# Calculate metrics
metrics = {}
for sentiment in sentiments:
    data = non_zero_data[non_zero_data['sentiment_group'] == sentiment]
    metrics[sentiment] = {
        'total_pnl': data['Closed PnL'].sum(),
        'avg_pnl': data['Closed PnL'].mean(),
        'win_rate': (data['Closed PnL'] > 0).sum() / len(data) * 100 if len(data) > 0 else 0,
        'trade_count': len(data)
    }

print("✓ Data prepared. Starting visualizations...")

# ============================================================================
# VISUALIZATION 1: Performance Metrics Overview (4-panel dashboard)
# ============================================================================

fig1 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Total Profit/Loss by Sentiment', 
                    'Average PnL per Trade',
                    'Win Rate by Sentiment',
                    'Trading Volume by Sentiment'),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Panel 1: Total PnL
total_pnls = [metrics[s]['total_pnl'] for s in sentiments]
fig1.add_trace(
    go.Bar(
        x=sentiments,
        y=total_pnls,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'${v:,.0f}' for v in total_pnls],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Total PnL',
        showlegend=False
    ),
    row=1, col=1
)

# Panel 2: Average PnL
avg_pnls = [metrics[s]['avg_pnl'] for s in sentiments]
fig1.add_trace(
    go.Bar(
        x=sentiments,
        y=avg_pnls,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'${v:.2f}' for v in avg_pnls],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Avg PnL',
        showlegend=False
    ),
    row=1, col=2
)

# Panel 3: Win Rate
win_rates = [metrics[s]['win_rate'] for s in sentiments]
fig1.add_trace(
    go.Bar(
        x=sentiments,
        y=win_rates,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'{v:.1f}%' for v in win_rates],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Win Rate',
        showlegend=False
    ),
    row=2, col=1
)

# Add 50% reference line for win rate
fig1.add_hline(y=50, line_dash="dash", line_color="orange", line_width=2,
               annotation_text="50% Breakeven", annotation_position="right",
               row=2, col=1)

# Panel 4: Trade Count
trade_counts = [metrics[s]['trade_count'] for s in sentiments]
fig1.add_trace(
    go.Bar(
        x=sentiments,
        y=trade_counts,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'{v:,}' for v in trade_counts],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Trade Count',
        showlegend=False
    ),
    row=2, col=2
)

# Update layout
fig1.update_xaxes(title_text="", row=1, col=1)
fig1.update_xaxes(title_text="", row=1, col=2)
fig1.update_xaxes(title_text="", row=2, col=1)
fig1.update_xaxes(title_text="", row=2, col=2)

fig1.update_yaxes(title_text="Total PnL ($)", row=1, col=1, gridcolor='lightgray')
fig1.update_yaxes(title_text="Avg PnL per Trade ($)", row=1, col=2, gridcolor='lightgray')
fig1.update_yaxes(title_text="Win Rate (%)", row=2, col=1, range=[0, 100], gridcolor='lightgray')
fig1.update_yaxes(title_text="Number of Trades", row=2, col=2, gridcolor='lightgray')

fig1.update_layout(
    title_text="<b>Trading Performance by Market Sentiment</b>",
    title_font_size=20,
    title_x=0.5,
    height=800,
    showlegend=False,
    hovermode='x unified',
    plot_bgcolor='white'
)

fig1.show()
print("Visualization 1 created: Performance Metrics Overview")

# ============================================================================
# VISUALIZATION 2: Risk-Reward Paradox (Bearish Period Deep Dive)
# ============================================================================

bearish_data = non_zero_data[non_zero_data['sentiment_group'] == 'Bearish'].copy()
bearish_wins = bearish_data[bearish_data['Closed PnL'] > 0]['Closed PnL']
bearish_losses = bearish_data[bearish_data['Closed PnL'] < 0]['Closed PnL']

fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribution of PnL During Bearish Periods',
                    'Cumulative P&L During Bearish Periods'),
    horizontal_spacing=0.12
)

# Panel 1: Histogram of wins vs losses
fig2.add_trace(
    go.Histogram(
        x=bearish_wins,
        name='Winning Trades',
        marker=dict(color='#2ecc71', line=dict(color='black', width=1)),
        opacity=0.7,
        nbinsx=30
    ),
    row=1, col=1
)

fig2.add_trace(
    go.Histogram(
        x=bearish_losses,
        name='Losing Trades',
        marker=dict(color='#e74c3c', line=dict(color='black', width=1)),
        opacity=0.7,
        nbinsx=30
    ),
    row=1, col=1
)

# Add vertical line at zero
fig2.add_vline(x=0, line_dash="dash", line_color="black", line_width=2, row=1, col=1)

# Panel 2: Cumulative PnL
bearish_sorted = bearish_data.sort_values('Timestamp IST')
cumulative_pnl = bearish_sorted['Closed PnL'].cumsum()

fig2.add_trace(
    go.Scatter(
        x=list(range(len(cumulative_pnl))),
        y=cumulative_pnl,
        mode='lines+markers',
        name='Cumulative PnL',
        line=dict(color='#e74c3c', width=3),
        marker=dict(size=4),
        fill='tozeroy',
        fillcolor='rgba(231, 76, 60, 0.3)'
    ),
    row=1, col=2
)

# Add horizontal line at zero
fig2.add_hline(y=0, line_dash="dash", line_color="black", line_width=2, row=1, col=2)

# Add annotation for final value
final_pnl = cumulative_pnl.iloc[-1]
fig2.add_annotation(
    x=len(cumulative_pnl)-1,
    y=final_pnl,
    text=f"Final: ${final_pnl:,.0f}",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="black",
    ax=-80,
    ay=-40,
    bgcolor="yellow",
    bordercolor="black",
    borderwidth=2,
    font=dict(size=12, color="black", family="Arial Black"),
    row=1, col=2
)

# Update axes
fig2.update_xaxes(title_text="PnL ($)", row=1, col=1, gridcolor='lightgray')
fig2.update_xaxes(title_text="Trade Number (Chronological)", row=1, col=2, gridcolor='lightgray')
fig2.update_yaxes(title_text="Frequency", row=1, col=1, gridcolor='lightgray')
fig2.update_yaxes(title_text="Cumulative PnL ($)", row=1, col=2, gridcolor='lightgray')

# Add statistics as annotation
stats_text = (f"<b>Statistics:</b><br>"
              f"Winning Trades: {len(bearish_wins)} (73.1%)<br>"
              f"Avg Win: ${bearish_wins.mean():.2f}<br><br>"
              f"Losing Trades: {len(bearish_losses)} (26.9%)<br>"
              f"Avg Loss: ${bearish_losses.mean():.2f}<br><br>"
              f"Win/Loss Ratio: {abs(bearish_wins.mean() / bearish_losses.mean()):.2f}x")

fig2.add_annotation(
    xref="paper", yref="paper",
    x=0.48, y=0.95,
    text=stats_text,
    showarrow=False,
    bgcolor="wheat",
    bordercolor="black",
    borderwidth=2,
    font=dict(size=11),
    align="left"
)

fig2.update_layout(
    title_text="<b>The Risk-Reward Paradox: Why High Win Rate ≠ Profitability</b>",
    title_font_size=18,
    title_x=0.5,
    height=500,
    showlegend=True,
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified',
    plot_bgcolor='white'
)

fig2.show()
print("Visualization 2 created: Risk-Reward Paradox")

# ============================================================================
# VISUALIZATION 3: Trader Participation and Activity
# ============================================================================

# Calculate trader activity
trader_activity = {}
for sentiment in sentiments:
    sentiment_data = merged_data[merged_data['sentiment_group'] == sentiment]
    trader_activity[sentiment] = {
        'unique_traders': sentiment_data['Account'].nunique(),
        'total_trades': len(sentiment_data),
        'avg_trades': len(sentiment_data) / sentiment_data['Account'].nunique() if sentiment_data['Account'].nunique() > 0 else 0
    }

fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Active Traders by Sentiment Period',
                    'Trading Intensity (Avg Trades per Trader)'),
    horizontal_spacing=0.15
)

# Panel 1: Active traders
active_traders = [trader_activity[s]['unique_traders'] for s in sentiments]
percentages = [(v / 32) * 100 for v in active_traders]  # 32 total traders

fig3.add_trace(
    go.Bar(
        x=sentiments,
        y=active_traders,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'{v}<br>({p:.0f}%)' for v, p in zip(active_traders, percentages)],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Active Traders',
        showlegend=False
    ),
    row=1, col=1
)

# Panel 2: Avg trades per trader
avg_trades = [trader_activity[s]['avg_trades'] for s in sentiments]

fig3.add_trace(
    go.Bar(
        x=sentiments,
        y=avg_trades,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'{v:.0f}' for v in avg_trades],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Avg Trades',
        showlegend=False
    ),
    row=1, col=2
)

fig3.update_xaxes(title_text="", row=1, col=1)
fig3.update_xaxes(title_text="", row=1, col=2)
fig3.update_yaxes(title_text="Number of Active Traders", row=1, col=1, gridcolor='lightgray')
fig3.update_yaxes(title_text="Average Trades per Trader", row=1, col=2, gridcolor='lightgray')

fig3.update_layout(
    title_text="<b>Trader Participation Across Market Sentiments</b>",
    title_font_size=18,
    title_x=0.5,
    height=500,
    hovermode='x unified',
    plot_bgcolor='white'
)

fig3.show()
print("Visualization 3 created: Trader Participation")

# ============================================================================
# VISUALIZATION 4: Position Sizing and Buy/Sell Behavior
# ============================================================================

fig4 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Position Sizing by Sentiment',
                    'Buy vs Sell Behavior by Sentiment'),
    horizontal_spacing=0.15
)

# Panel 1: Position sizes
position_sizes = []
for sentiment in sentiments:
    sentiment_data = merged_data[merged_data['sentiment_group'] == sentiment]
    avg_size = sentiment_data['Size USD'].mean()
    position_sizes.append(avg_size)

fig4.add_trace(
    go.Bar(
        x=sentiments,
        y=position_sizes,
        marker=dict(color=[colors_dict[s] for s in sentiments],
                   line=dict(color='black', width=2)),
        text=[f'${v:,.0f}' for v in position_sizes],
        textposition='outside',
        textfont=dict(size=12, color='black', family='Arial Black'),
        name='Position Size',
        showlegend=False
    ),
    row=1, col=1
)

# Add annotation for counter-intuitive pattern
fig4.add_annotation(
    x='Neutral',
    y=position_sizes[1],
    text="Largest positions<br>during neutral!",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="red",
    ax=60,
    ay=-60,
    bgcolor="yellow",
    bordercolor="red",
    borderwidth=2,
    font=dict(size=11, color="red", family="Arial Black"),
    row=1, col=1
)

# Panel 2: Buy vs Sell
buy_sell_data = []
for sentiment in sentiments:
    sentiment_data = merged_data[merged_data['sentiment_group'] == sentiment]
    buy_pct = (sentiment_data['Side'] == 'BUY').sum() / len(sentiment_data) * 100
    sell_pct = (sentiment_data['Side'] == 'SELL').sum() / len(sentiment_data) * 100
    buy_sell_data.append({'sentiment': sentiment, 'BUY': buy_pct, 'SELL': sell_pct})

# BUY bars
fig4.add_trace(
    go.Bar(
        x=sentiments,
        y=[d['BUY'] for d in buy_sell_data],
        name='BUY',
        marker=dict(color='#3498db', line=dict(color='black', width=1.5)),
        text=[f"{d['BUY']:.1f}%" for d in buy_sell_data],
        textposition='inside',
        textfont=dict(size=11, color='white', family='Arial Black')
    ),
    row=1, col=2
)

# SELL bars
fig4.add_trace(
    go.Bar(
        x=sentiments,
        y=[d['SELL'] for d in buy_sell_data],
        name='SELL',
        marker=dict(color='#e67e22', line=dict(color='black', width=1.5)),
        text=[f"{d['SELL']:.1f}%" for d in buy_sell_data],
        textposition='inside',
        textfont=dict(size=11, color='white', family='Arial Black')
    ),
    row=1, col=2
)

# Add 50% reference line
fig4.add_hline(y=50, line_dash="dash", line_color="black", line_width=1, 
               annotation_text="50% Line", row=1, col=2)

fig4.update_xaxes(title_text="", row=1, col=1)
fig4.update_xaxes(title_text="", row=1, col=2)
fig4.update_yaxes(title_text="Average Position Size ($)", row=1, col=1, gridcolor='lightgray')
fig4.update_yaxes(title_text="Percentage of Trades (%)", row=1, col=2, range=[0, 100], gridcolor='lightgray')

fig4.update_layout(
    title_text="<b>Position Sizing and Trading Behavior by Sentiment</b>",
    title_font_size=18,
    title_x=0.5,
    height=500,
    barmode='stack',
    hovermode='x unified',
    plot_bgcolor='white',
    legend=dict(x=0.85, y=0.5)
)

fig4.show()
print("Visualization 4 created: Position Sizing and Behavior")

# ============================================================================
# VISUALIZATION 5: Top Coins by Sentiment (Interactive)
# ============================================================================

fig5 = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Bearish Period', 'Neutral Period', 'Bullish Period'),
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]],
    horizontal_spacing=0.08
)

for idx, sentiment in enumerate(sentiments):
    sentiment_data = merged_data[merged_data['sentiment_group'] == sentiment]
    top_coins = sentiment_data['Coin'].value_counts().head(10)
    
    fig5.add_trace(
        go.Bar(
            y=top_coins.index[::-1],  # Reverse for top-to-bottom
            x=top_coins.values[::-1],
            orientation='h',
            marker=dict(color=colors_dict[sentiment],
                       line=dict(color='black', width=1)),
            text=[f'{v:,}' for v in top_coins.values[::-1]],
            textposition='outside',
            textfont=dict(size=10),
            name=sentiment,
            showlegend=False,
            hovertemplate='<b>%{y}</b><br>Trades: %{x:,}<extra></extra>'
        ),
        row=1, col=idx+1
    )
    
    fig5.update_xaxes(title_text="Number of Trades", row=1, col=idx+1, gridcolor='lightgray')
    fig5.update_yaxes(title_text="", row=1, col=idx+1)

fig5.update_layout(
    title_text="<b>Top Traded Coins by Market Sentiment</b>",
    title_font_size=18,
    title_x=0.5,
    height=500,
    hovermode='closest',
    plot_bgcolor='white'
)

fig5.show()
print("Visualization 5 created: Top Coins by Sentiment")

# ============================================================================
# VISUALIZATION 6: Executive Dashboard (All-in-One)
# ============================================================================

fig6 = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Total PnL', 'Win Rate', 'Active Traders',
                    'Avg PnL per Trade', 'Position Size', 'Trade Volume',
                    '', '', ''),
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
           [{"type": "table", "colspan": 3}, None, None]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1,
    row_heights=[0.3, 0.3, 0.4]
)

# Row 1, Col 1: Total PnL
fig6.add_trace(
    go.Bar(x=sentiments, y=total_pnls,
           marker=dict(color=[colors_dict[s] for s in sentiments], line=dict(color='black', width=1.5)),
           text=[f'${v:,.0f}' for v in total_pnls], textposition='outside',
           showlegend=False),
    row=1, col=1
)

# Row 1, Col 2: Win Rate
fig6.add_trace(
    go.Bar(x=sentiments, y=win_rates,
           marker=dict(color=[colors_dict[s] for s in sentiments], line=dict(color='black', width=1.5)),
           text=[f'{v:.1f}%' for v in win_rates], textposition='outside',
           showlegend=False),
    row=1, col=2
)

# Row 1, Col 3: Active Traders
fig6.add_trace(
    go.Bar(x=sentiments, y=active_traders,
           marker=dict(color=[colors_dict[s] for s in sentiments], line=dict(color='black', width=1.5)),
           text=active_traders, textposition='outside',
           showlegend=False),
    row=1, col=3
)

# Row 2, Col 1: Avg PnL
fig6.add_trace(
    go.Bar(x=sentiments, y=avg_pnls,
           marker=dict(color=[colors_dict[s] for s in sentiments], line=dict(color='black', width=1.5)),
           text=[f'${v:.2f}' for v in avg_pnls], textposition='outside',
           showlegend=False),
    row=2, col=1
)

# Row 2, Col 2: Position Size
fig6.add_trace(
    go.Bar(x=sentiments, y=position_sizes,
           marker=dict(color=[colors_dict[s] for s in sentiments], line=dict(color='black', width=1.5)),
           text=[f'${v:,.0f}' for v in position_sizes], textposition='outside',
           showlegend=False),
    row=2, col=2
)

# Row 2, Col 3: Trade Volume
fig6.add_trace(
    go.Bar(x=sentiments, y=trade_counts,
           marker=dict(color=[colors_dict[s] for s in sentiments], line=dict(color='black', width=1.5)),
           text=[f'{v:,}' for v in trade_counts], textposition='outside',
           showlegend=False),
    row=2, col=3
)

# Row 3: Summary Table
summary_data = pd.DataFrame({
    'Sentiment': sentiments,
    'Total PnL': [f"${metrics[s]['total_pnl']:,.0f}" for s in sentiments],
    'Avg PnL/Trade': [f"${metrics[s]['avg_pnl']:.2f}" for s in sentiments],
    'Win Rate': [f"{metrics[s]['win_rate']:.1f}%" for s in sentiments],
    'Trades': [f"{metrics[s]['trade_count']:,}" for s in sentiments],
    'Active Traders': [trader_activity[s]['unique_traders'] for s in sentiments]
})

fig6.add_trace(
    go.Table(
        header=dict(values=list(summary_data.columns),
                   fill_color='lightgray',
                   align='center',
                   font=dict(size=12, color='black', family='Arial Black')),
        cells=dict(values=[summary_data[col] for col in summary_data.columns],
                  fill_color=[['#e74c3c', '#95a5a6', '#2ecc71']*len(summary_data.columns)],
                  align='center',
                  font=dict(size=11, color='white', family='Arial'),
                  height=30)
    ),
    row=3, col=1
)

# Update axes
fig6.update_yaxes(title_text="$ Amount", row=1, col=1, gridcolor='lightgray')
fig6.update_yaxes(title_text="%", row=1, col=2, gridcolor='lightgray')
fig6.update_yaxes(title_text="Count", row=1, col=3, gridcolor='lightgray')
fig6.update_yaxes(title_text="$ Amount", row=2, col=1, gridcolor='lightgray')
fig6.update_yaxes(title_text="$ Amount", row=2, col=2, gridcolor='lightgray')
fig6.update_yaxes(title_text="Count", row=2, col=3, gridcolor='lightgray')

fig6.update_layout(
    title_text="<b>Market Sentiment Analysis - Executive Dashboard</b>",
    title_font_size=22,
    title_x=0.5,
    height=1000,
    showlegend=False,
    hovermode='x unified',
    plot_bgcolor='white'
)

fig6.show()
print("Visualization 6 created: Executive Dashboard")



✓ Data prepared. Starting visualizations...


Visualization 1 created: Performance Metrics Overview


Visualization 2 created: Risk-Reward Paradox


Visualization 3 created: Trader Participation


Visualization 4 created: Position Sizing and Behavior


Visualization 5 created: Top Coins by Sentiment


Visualization 6 created: Executive Dashboard


# Market Sentiment and Trader Performance Analysis
## Exploring the Relationship Between Bitcoin Fear & Greed Index and Hyperliquid Trading Performance

---

## Executive Summary

This analysis examines 52,497 trades from 32 traders on the Hyperliquid platform throughout 2024, analyzing their performance against Bitcoin's Fear & Greed Index. The findings reveal a **strong correlation between market sentiment and trading profitability**, with critical implications for risk management and trading strategy.

**Key Takeaway:** Traders are significantly more profitable during bullish (greed) periods, while bearish (fear) periods result in net losses despite high win rates—a classic risk-reward inversion pattern.

---

## Dataset Overview

### Data Scope
- **Time Period:** January 1, 2024 - December 31, 2024
- **Total Trades:** 52,497 trades across 341 active trading days
- **Traders:** 32 unique accounts
- **Coins Traded:** 246 different cryptocurrencies
- **Sentiment Coverage:** 365 days of Bitcoin Fear & Greed Index data

### Sentiment Distribution (2024)
- **Bullish (Greed + Extreme Greed):** 263 days (72.1%)
- **Bearish (Fear + Extreme Fear):** 59 days (16.2%)
- **Neutral:** 43 days (11.8%)

**Note:** 2024 was predominantly a bullish year, with greed sentiment dominating the market landscape.

---

## Key Findings

### Finding 1: Sentiment Dramatically Impacts Profitability

#### Overall Performance by Sentiment

| Sentiment | Total Trades | Non-Zero PnL Trades | Win Rate | Total PnL | Avg PnL/Trade |
|-----------|-------------|---------------------|----------|-----------|---------------|
| **Bearish** | 1,451 | 475 | 73.1% | -$87,196 | -$183.57 |
| **Neutral** | 1,293 | 559 | 55.1% | +$88,120 | +$157.64 |
| **Bullish** | 49,747 | 26,066 | 84.6% | +$3,563,464 | +$136.71 |

**Critical Insight:** Despite having a 73.1% win rate during bearish periods, traders still lose money on average. This represents a dangerous **risk-reward inversion** where small wins are obliterated by occasional catastrophic losses.

#### Profitability Analysis
- **Bullish periods generate 98% of total profits** ($3.56M out of $3.56M net)
- **Bearish periods result in net losses** despite winning 73% of trades
- **Average loss per losing trade during fear is significantly larger** than average win per winning trade

---

### Finding 2: The "Catching Falling Knives" Phenomenon

During bearish (fear) periods, traders exhibit classic behavioral finance patterns:

**The Paradox:**
- ✓ High win rate (73.1%)
- ✗ Negative returns (-$183.57 avg)

**What's happening:**
- Traders successfully capture small rebounds (many small wins)
- But fail to manage risk on major drawdowns (few massive losses)
- Risk-reward ratio is inverted: winners average ~$50-100, losers can exceed $1,000+

**Example Pattern:** A trader might win $50 on 7 out of 10 trades (+$350), but lose $500 on each of the 3 losing trades (-$1,500), resulting in a net loss of -$1,150 despite a 70% win rate.

---

### Finding 3: Trader Participation Varies Dramatically by Sentiment

#### Trader Activity by Sentiment

| Sentiment | Active Traders | Total Trades | Trades/Trader | % of Total Volume |
|-----------|---------------|--------------|---------------|-------------------|
| **Bearish** | 7 traders | 1,451 | 207 | 2.8% |
| **Neutral** | 9 traders | 1,293 | 144 | 2.5% |
| **Bullish** | 31 traders | 49,747 | 1,605 | 94.7% |

**Trader Segmentation:**
- **"All-Weather" Traders:** 6 traders active across all sentiment periods (19% of total)
- **"Fair-Weather" Traders:** 21 traders ONLY active during bullish periods (68% of total)
- **Sporadic Traders:** 5 traders with limited activity

**Behavioral Insight:** The majority of traders (68%) deliberately avoid or significantly reduce activity during bearish periods, suggesting an awareness of the difficulty in profiting during fear-driven markets.

---

### Finding 4: Position Sizing and Trading Behavior Shifts

#### Average Position Size by Sentiment

| Sentiment | Avg Position Size | Behavior |
|-----------|------------------|----------|
| **Bearish** | $4,574 | Medium-sized, cautious entries |
| **Neutral** | $6,900 | **LARGEST positions** (highest conviction) |
| **Bullish** | $3,324 | **SMALLEST positions** (risk management) |

**Counter-Intuitive Discovery:** Traders use their SMALLEST position sizes during the most profitable period (bullish), suggesting sophisticated risk management—taking many small profitable trades rather than large concentrated bets.

#### Buy/Sell Distribution

| Sentiment | % BUY | % SELL | Interpretation |
|-----------|-------|--------|----------------|
| **Bearish** | 56.3% | 43.7% | Attempting to buy dips (contrarian) |
| **Neutral** | 45.3% | 54.7% | Balanced approach |
| **Bullish** | 44.0% | 56.0% | Taking profits, selling into strength |

**Strategy Observation:** Traders exhibit contrarian tendencies (buy fear, sell greed), but their execution during fear periods results in losses, suggesting poor timing or inadequate risk management.

---

### Finding 5: Coin Selection Changes with Market Sentiment

#### Top 3 Coins Traded by Sentiment

| Sentiment | Top Coins | Characteristics |
|-----------|-----------|-----------------|
| **Bearish** | ETH, @4, @8 | Major coins + unknown tokens |
| **Neutral** | ETH, BTC, @4 | Blue-chip cryptocurrencies |
| **Bullish** | @107, HYPE, ETH | Meme coins and trending tokens |

**Key Observations:**
- During greed periods, traders shift toward **high-volatility meme coins** (HYPE, @107)
- During fear periods, traders prefer **established major coins** (ETH, BTC)
- **HYPE coin alone represents 32% of all trades**, concentrated during bullish periods

**Profitability Note:** The shift to meme coins during bullish periods appears profitable, as these assets exhibit stronger momentum during greed phases.

---

### Finding 6: The Zero-PnL Mystery

**48.4% of all trades (25,391 trades) have zero PnL**, indicating:
- Position openings that haven't been closed yet
- The dataset captures both entry and exit transactions
- Only 51.6% of trades represent completed positions with realized PnL

**Adjusted Performance Metrics:**
- When analyzing only closed positions (non-zero PnL), the true win/loss dynamics become clear
- Win rates are significantly higher than initially apparent
- But loss magnitude during bearish periods still dominates

---

## Strategic Implications

### For Individual Traders

1. **Avoid Active Trading During Extreme Fear**
   - Historical data shows net losses despite high win rates
   - Risk-reward is structurally unfavorable
   - Better to wait for neutral/bullish conditions

2. **Implement Strict Stop-Losses During Bearish Periods**
   - The data shows small wins and large losses
   - Position sizing must account for potential cascading losses
   - Consider reducing leverage or position size by 50%+

3. **Capitalize on Bullish Momentum**
   - 84.6% win rate and positive average PnL
   - High-frequency trading with smaller positions works
   - Meme coins and trending tokens outperform during this phase

4. **Adopt Counter-Cyclical Position Sizing**
   - Reduce position sizes during greed (protect gains)
   - Increase caution during fear (manage tail risk)

### For Risk Managers

1. **Sentiment-Based Portfolio Allocation**
   - Reduce capital allocation during extended bearish periods
   - Increase allocation during confirmed bullish trends
   - Monitor sentiment transitions for rebalancing signals

2. **Volatility Adjustments**
   - Expect higher loss volatility during fear periods
   - Plan for drawdowns exceeding 2-3x normal during sentiment shifts

---

## Limitations and Considerations

1. **Sample Size Imbalance:** Bearish periods represent only 2.8% of total trades, which may limit statistical significance of bearish-period findings.

2. **Survivorship Bias:** The dataset only includes traders who remained active through 2024. Traders who were eliminated or quit are not represented.

3. **Temporal Causality:** While sentiment correlates with performance, we cannot definitively prove causation. Market conditions beyond sentiment (liquidity, volatility, news events) may be confounding factors.

4. **Platform-Specific Behavior:** These patterns are specific to Hyperliquid traders and may not generalize to other platforms or trader populations.

5. **2024 Bias:** This was a predominantly bullish year (72% greed days). Results may differ in bear-market years with inverted sentiment distribution.

6. **Sentiment Lag:** The Fear & Greed Index is a lagging indicator. By the time "extreme fear" or "extreme greed" is registered, the optimal entry/exit point may have passed.

---

## Conclusions

This analysis reveals a **clear and actionable relationship between market sentiment and trading performance**:

✅ **Bullish sentiment periods are highly profitable** (84.6% win rate, +$136 avg PnL)  
✅ **Most traders recognize this** and concentrate activity during greed  
❌ **Bearish sentiment periods are unprofitable** despite high win rates (risk-reward inversion)  
❌ **Trading during fear requires exceptional risk management** that most traders lack  

**The Winning Strategy:** The data suggests that the most successful approach is not to trade counter-cyclically during fear, but rather to **reduce activity during bearish periods and capitalize aggressively during bullish trends**—contrary to the classic "be fearful when others are greedy" advice.

The 21 traders who only operate during bullish periods may actually be employing the most rational strategy based on historical profitability data.

